# Kernel Semantik

Dalam contoh kod ini, anda akan menggunakan rangka kerja AI [Kernel Semantik](https://aka.ms/ai-agents-beginners/semantic-kernel) untuk mencipta ejen asas.

Matlamat contoh ini adalah untuk menunjukkan kepada anda langkah-langkah yang akan kita gunakan kemudian dalam contoh kod tambahan apabila melaksanakan corak ejen yang berbeza.


## Import Pakej Python yang Diperlukan


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Membuat Klien

Dalam contoh ini, kita akan menggunakan [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) untuk mengakses LLM.

`ai_model_id` ditetapkan sebagai `gpt-4o-mini`. Cuba tukar model kepada model lain yang tersedia di pasaran GitHub Models untuk melihat hasil yang berbeza.

Untuk menggunakan `Azure Inference SDK` yang digunakan untuk `base_url` bagi GitHub Models, kita akan menggunakan penyambung `OpenAIChatCompletion` dalam Semantic Kernel. Terdapat juga [penyambung lain yang tersedia](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) untuk menggunakan Semantic Kernel dengan penyedia model lain.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Membuat Ejen

Di bawah ini, kita akan membuat Ejen yang dipanggil `TravelAgent`.

Untuk contoh ini, kita menggunakan arahan yang sangat mudah. Anda boleh mengubah arahan ini untuk melihat bagaimana ejen memberikan respons yang berbeza.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Menjalankan Ejen

Sekarang kita boleh menjalankan Ejen dengan mendefinisikan thread jenis `ChatHistoryAgentThread`. Sebarang mesej sistem yang diperlukan disediakan kepada argumen kata kunci `messages` dalam invoke_stream ejen.

Selepas ini ditetapkan, kita mencipta `user_inputs` yang akan menjadi apa yang dihantar oleh pengguna kepada ejen. Dalam kes ini, kita telah menetapkan mesej ini kepada `Plan me a sunny vacation`.

Jangan ragu untuk menukar mesej ini untuk melihat bagaimana ejen memberikan respons yang berbeza.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
